In [2]:
from dash import Dash
import pandas as pd

# I fixed your imports for dash.dcc and dash.html,  the reason you were getting a warning message was because the way you imported the library worked 
# but was an outdated method of doing it

from dash import dcc,html
import plotly.express as px
from dash.dependencies import Input, Output
import pymssql
import plotly.graph_objects as go


import warnings # The method used in class gives a warning when using pd.read_sql, this will ignore the warning message
warnings.simplefilter(action='ignore', category=UserWarning)

database = "Pushing-P-DB"
table = "dbo.NJ_Cities"
user = "pushing_p"
password  = "t3stP@ssword"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"


# Make function to retrieve data from database
def Get_Data():
    try:
        conn = pymssql.connect(server,user,password,database)

        query = f"SELECT * FROM {table}"

        df = pd.read_sql(query, conn)

        # converting to pandas dataframe
        df = pd.DataFrame(df)
        df['Rate Of Uninsured'] = df['Uninsured Population']/df['City Population']*100
        df['Rate Of Uninsured'] = df['Rate Of Uninsured'].round(decimals = 3)

        # This is where I format the TimeStamp to "ms" and then I have to for some reason use ".dt.strftime('%Y-%m-%d %H:%M:%S')" because the format in the table chances when I print it out
#         df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')

        # This is where I order based off of timestamp for my fifth dataframe
#         df = df.sort_values(by='TimeStamp',ascending=False)

        return df

    except Exception as e:
        raise e


In [3]:
df = Get_Data()
df

,State,County,City,Unemployment Rate (16 & Over),Median Household Income,City Population,Uninsured Population,"NUI, Under 6 years","NUI, 6-18 years","NUI, 19 to 25 years",...,"NUI, Men","NUI, Women","NUI, White","NUI, African American","NUI, American Indian & Alaska Native Population","NUI, Asians","NUI, Native Hawaiians & Pacific Islanders Population","NUI, Other Races",geometry,Rate Of Uninsured
0,New Jersey,Burlington County,Riverside Township,9.8,61975,7836,1409,17,85,202,...,805,604,821,154,28,13,0,236,01030000000100000048000000B60F79CBD5B752C0F182...,17.981
1,New Jersey,Monmouth County,Manalapan Township,4.1,130663,39462,950,0,114,101,...,402,548,734,28,0,143,0,17,010300000002000000650000008E75711B0D9C52C00725...,2.407
2,New Jersey,Monmouth County,Manasquan Borough,4.9,116435,5837,113,0,17,0,...,39,74,113,0,0,0,0,0,01030000000100000015000000641F645930B352C019E1...,1.936
3,New Jersey,Burlington County,Riverton Borough,5.1,100278,2592,75,12,0,0,...,63,12,42,14,0,0,0,19,01030000000100000045000000B64604E3E0B452C0D445...,2.894
4,New Jersey,Monmouth County,Fair Haven Borough,3.2,184201,5833,122,0,34,0,...,60,62,121,1,0,0,0,0,01030000000100000015000000C2BF081A338052C062BA...,2.092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,New Jersey,Bergen County,Englewood,5.9,86485,28213,2735,121,407,267,...,1257,1478,1514,698,34,17,0,411,0103000000010000002D000000C26B97361C8052C01616...,9.694
561,New Jersey,Bergen County,Englewood Cliffs Borough,4.6,158042,5349,252,0,9,0,...,104,148,138,0,0,92,10,0,010300000001000000310000001F63EE5A42A852C063B3...,4.711
562,New Jersey,Monmouth County,Englishtown Borough,9.0,84091,1999,51,0,0,10,...,21,30,25,16,0,0,0,0,0103000000010000002E000000471E882CD29952C0FF76...,2.551
563,New Jersey,Essex County,Essex Fells Borough,3.1,215104,2037,15,0,4,0,...,4,11,15,0,0,0,0,0,01030000000100000017000000B06EBC3B32C452C01022...,0.736


In [4]:
df.columns

Index(['State', 'County', 'City', 'Unemployment Rate (16 & Over)',
       'Median Household Income', 'City Population', 'Uninsured Population',
       'NUI, Under 6 years', 'NUI, 6-18 years', 'NUI, 19 to 25 years',
       'NUI, 26 to 34 years', 'NUI, 35 to 44 years', 'NUI, 45 to 54 years',
       'NUI, 55 to 64 years', 'NUI, 65 years and older', 'NUI, Men',
       'NUI, Women', 'NUI, White', 'NUI, African American',
       'NUI, American Indian & Alaska Native Population', 'NUI, Asians',
       'NUI, Native Hawaiians & Pacific Islanders Population',
       'NUI, Other Races', 'geometry', 'Rate Of Uninsured'],
      dtype='object')

In [5]:
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque




In [6]:
df = df[['Uninsured Population','NUI, Men','NUI, Women']]
Male_sum = df['NUI, Men'].sum()
Female_sum = df["NUI, Women"].sum()
total_uninsured = df['Uninsured Population'].sum()
type(total_uninsured)



numpy.int64

In [21]:
X = deque(maxlen = 20)
X.append(0)


Y1 = deque(maxlen = 20)
Y1.append(0)


Y2 = deque(maxlen = 20)
Y2.append(0)


app = dash.Dash(__name__)

app.layout = html.Div(
	[
		dcc.Graph(id = 'live-graph', animate = True),
		dcc.Interval(
			id = 'graph-update',
			interval = 10000,
			n_intervals = 0,
		),
	]
)

@app.callback(
	Output('live-graph', 'figure'),
	[ Input('graph-update', 'n_intervals') ]
)

def update_graph_scatter(n):
	global total_uninsured
	global Male_sum
	global Female_sum
	if (X[-1] - X[-2]) != total_uninsured:
		total_uninsured += total_uninsured
	if (Y1[-1] - Y1[-2]) != Male_sum:
		Male_sum += Male_sum
	if (Y2[-1] - Y2[-2]) != Female_sum:
		Female_sum += Female_sum
	
	X.append(total_uninsured)
	Y1.append(Male_sum)
	Y2.append(Female_sum)
	

	data = plotly.graph_objs.Scatter(
			x=list(X),
			y=list(Y1),
			name='Male',
			mode= 'lines+markers'
	        )
            
	data1 = plotly.graph_objs.Scatter(
			x=list(X),
			y=list(Y2),
			name='Female',
			mode= 'lines+markers'
	        )      

	return {'data': [data,data1],
			'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),yaxis = dict(range = [min(Y1),max(Y1)]),)}

if __name__ == '__main__':
	app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/May/2022 10:40:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:47] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2022 10:40:47]

In [26]:
X[-1]

339413504

In [11]:
Y1

deque([363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100,
       363518,
       100])

deque([299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75,
       299399,
       75])